In [ ]:
!pip install -q simpletransformers

     |████████████████████████████████| 247 kB 12.1 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 9.1 MB 43.3 MB/s 
     |████████████████████████████████| 290 kB 49.0 MB/s 
     |████████████████████████████████| 1.7 MB 24.0 MB/s 
     |████████████████████████████████| 1.2 MB 50.8 MB/s 
     |████████████████████████████████| 3.1 MB 42.1 MB/s 
     |████████████████████████████████| 3.3 MB 41.4 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 895 kB 39.9 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 180 kB 47.8 MB/s 
     |████████████████████████████████| 140 kB 45.6 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 132 kB 50.1 MB/s 
     |████████████████████████████████| 243 kB 51.6 MB/s 
     |████████████████

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=3)

In [ ]:
data = pd.read_csv('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/twitter_hate.zip')

In [ ]:
data_df = pd.DataFrame({'text':data['tweet'], 'labels':data['class']})

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(data_df, data_df['labels'])

In [ ]:
# module to split data into training / test
from sklearn.model_selection import train_test_split

In [ ]:
train_df, eval_df = train_test_split(data_df_res, test_size=0.2, random_state=18)

In [ ]:
!rm -rf outputs

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "train_batch_size": 64,
    "num_train_epochs": 4,
    "save_eval_checkpoints": False,
    #"mlm": False,
    #"best_model_dir": "/content/drive/MyDrive/Colab/GPR2Elon/best" #<-- after mounting google drive
    "learning_rate" : 5e-5
}

# Optional model configuration
model_args = ClassificationArgs()

# Create a ClassificationModel
model = ClassificationModel(
    'distilbert',
    'distilbert-base-uncased',
    num_labels=3,
    #args=model_args, 
    use_cuda=True,
    args=train_args
) 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
# Train the model
model.train_model(train_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3432 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_distilbert_128_3_2


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/108 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/108 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/108 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.


(324, 0.6631051934795615)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df.sample(100))



INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_3_2


Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7905609980557391, 'eval_loss': 0.4874710669884315}


In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(eval_df.text.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/858 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(eval_df.labels, predictions, target_names=['hate','offensive','nix']))

              precision    recall  f1-score   support

        hate       0.76      0.72      0.74       304
   offensive       0.78      0.77      0.77       274
         nix       0.85      0.90      0.87       280

    accuracy                           0.79       858
   macro avg       0.79      0.80      0.79       858
weighted avg       0.79      0.79      0.79       858



In [ ]:
# Make predictions with the model

non_balanced_sample = data_df.sample(2000)

predictions_nb, raw_outputs_nb = model.predict(non_balanced_sample.text.tolist())



INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
print(classification_report(non_balanced_sample.labels, predictions_nb, target_names=['hate','offensive','nix']))

              precision    recall  f1-score   support

        hate       0.22      0.70      0.34       115
   offensive       0.97      0.80      0.88      1550
         nix       0.84      0.90      0.87       335

    accuracy                           0.81      2000
   macro avg       0.68      0.80      0.70      2000
weighted avg       0.91      0.81      0.85      2000

